## Code Summary

## 1. Semi-analytical model and data generation

### 1.1 Magnetic force modelling

In [ ]:
# Importing relevant libraries

import matplotlib as plt
import math as m
from numpy import sqrt, sin, cos, pi
from scipy.integrate import quad

# Delta funcion
def delta(theta,alpha,r):
    return 2*r*cos(alpha-theta)

# Pxi plus
def pxi_plus(r,z,h):
    return r**2 + (z-h)**2

# Pxi minus
def pxi_minus(r,z,h):
    return r**2 + z**2

# Mag function
def mag(J):
    return J/(4*pi)

# Auxiliary radius
def r0(theta,R):
    a = R
    b = R
    return a*b/sqrt(a**2*sin(theta)**2 + b**2*cos(theta)**2)

# Computation of the magnetic field from the upper (plus) surface (dB_plus)
def dB_plus(theta,r, alpha, z, R, h):
    db_func = (2*(delta(theta,alpha,r)*r0(theta,R) - 
                  2*pxi_plus(r,z,h))/(4*pxi_plus(r,z,h) - 
                                      delta(theta,alpha,r)**2)/sqrt((r0(theta,R)*
                                                                     (r0(theta,R)-delta(theta,alpha,r))+
                                                                     pxi_plus(r,z,h))) + 4*sqrt(pxi_plus(r,z,h))/\
               (4*pxi_plus(r,z,h)-delta(theta,alpha,r)**2))*(z-h)
    return db_func
    
# Computation of the magnetic field from the lower (minus) surface (dB_minus)
def dB_minus(theta,r, alpha, z, R, h):
    db_func = -(2*(delta(theta,alpha,r)*r0(theta,R) - 
                  2*pxi_minus(r,z,h))/(4*pxi_minus(r,z,h) - 
                                      delta(theta,alpha,r)**2)/sqrt((r0(theta,R)*
                                                                     (r0(theta,R)-delta(theta,alpha,r))+
                                                                     pxi_minus(r,z,h))) + 4*sqrt(pxi_minus(r,z,h))/\
                (4*pxi_minus(r,z,h)-delta(theta,alpha,r)**2))*(z)
    return db_func

# Computation of the total magnetic field from both surfaces (dB)
def dB(theta,r, alpha, z, R, h):
    return dB_plus(theta,r, alpha, z, R, h) + dB_minus(theta,r, alpha, z, R, h)

# Semi-analytical model for magnetic force computation (triple_db)

def triple_db(r_s,h1, pxi, R, h):
    return tplquad(lambda theta,r,alpha: r*dB(theta,r, alpha, pxi+h, R, h),
                   -pi,pi, lambda r: 0, lambda r: r_s, lambda r,alpha: -pi, lambda r,alpha: pi)[0] - \
tplquad(lambda theta,r,alpha: r*dB(theta,r, alpha, pxi+h+h1, R, h),-pi,pi, lambda r: 0, lambda r: r_s, 
        lambda r,alpha: -pi, lambda r,alpha: pi)[0]

### 1.2 Data generation

The random numbers generated in this study following the Mersenne Twister generator [1].<br>
[1] M. Matsumoto and T. Nishimura, “Mersenne Twister: A 623-dimensionally equidistributed uniform pseudorandom number generator”, ACM Transactions on Modeling and Computer Simulation Vol. 8, No. 1, January pp.3–30 1998.

In [ ]:
# Importing relevant libraries

import numpy as np
import random
import pickle
from scipy.integrate import tplquad

# Defining the constants
mu0 = 4*pi*10**-7 # the magnetic permeability
K = 1/4/pi/mu0 # auxiliary constant

# Creating emty lists
mF_r = [] # Magnetic force empty list
para_r = [] # Empty list of parameters

# Generating random parameters
for item in range(116448):
    R = random.randint(3,30)/1000 # Radius of upper magnet
    h1 = random.randint(5,50)/1000 # Thickness of upper magnet
    r_s = random.randint(3,30)/1000 # Radius of lower magnet
    h = random.randint(5,50)/1000 # Thickness of lower magnet
    pxi = random.randint(2,50)/1000 # Separation distance between two magnets
    mF_r.append(K*triple_db(r_s,h1, pxi, R, h)) # Magnetic force computation and appending to mF_r list
    para_r.append([R*1000,h1*1000,r_s*1000,h*1000,pxi*1000]) # Appending parameters to para_r list
    
# Saving generated data

saved_file = open('mF_rr', 'wb') # mF_r is saved in mF_rr file
pickle.dump(mF_r, saved_file)

saved_file = open('para_rr', 'wb') # para_r is saved in para_rr file
pickle.dump(para_rr, saved_file)

## 2. Data preprocessing

### 2.1 Loading data

In [175]:
# Importing relevant libraries
import pickle

# Loading magnetic force
pick_in = open('mF_rr','rb')
mF__r = pickle.load(pick_in)
pick_in.close()

# Loading parameters
pick_in1 = open('para_rr','rb')
para__r = pickle.load(pick_in1)
pick_in.close()

### 2.2 Cleansing data

In [176]:
# Cleansing those parameters created the magnetic force less than or equal to 0.0134 (empirical)
i = 0
while i < len(mF__r):
    if mF__r[i] <= 0.0134:
        mF__r.pop(i)
        para__r.pop(i)
        i -= 1
    i += 1   
print(f'len: {len(mF__r)}, min: {min(mF__r)}')

len: 116361, min: 0.013400648488270193


In [177]:
# Convert float 64 into float 32
import numpy as np

mF__r = np.array(mF__r, dtype = np.float32)
para__r = np.array(para__r, dtype = np.float32)

### 2.3 Feature engineering

In [178]:
# Importing relevant libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assigning data

X = para__r
y = mF__r

# Creating training and testing data
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.08,shuffle = True)

# Creating traning and validation data
X_test, X_valid, y_test, y_valid = train_test_split(X_test_val, y_test_val, test_size=0.5,shuffle = True)

# Scaling data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

## 3. Deep learning model building

In [179]:
# Importing relevant libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Bulding model
model = Sequential()
model.add(Dense(300,activation = 'relu'))
model.add(Dense(300,activation = 'relu'))
model.add(Dense(300,activation = 'relu'))
model.add(Dense(300,activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

## 4. Model training

In [ ]:
# Importing relevant libraries
import time
from matplotlib import pyplot as plt
%matplotlib inline

# Creating empty lists
test_error =[]
loss = []
val_loss = []
training_error =1
val_error = 1
index = 0
test_los = 1

# Time counting initiated
start_time = time.clock()

# Training process
while (training_error > 0.065**2) or (test_los > 0.065**2):
    model.fit(x=X_train, y=y_train, batch_size = 1024, validation_data=(X_valid,y_valid), verbose=1, shuffle=True)
    test_error.append(model.evaluate(X_test, y_test))
    loss.append(model.history.history['loss'][0])
    val_loss.append(model.history.history['val_loss'][0])
    index +=1
    training_error = loss[index-1]
    val_error = val_loss[index-1]
    test_los = test_error[index-1]                                                              

# Training time computation
trained_time = time.clock() - start_time    

# Visualizing training errors
error_ = {'Training': loss, 'Validation': val_loss, 'Test': test_error}
plt.plot(loss,label = 'Training',ls='--')
plt.plot(val_loss,label = 'Validation',marker='o')
plt.plot(test_error, label = 'Test')
plt.xlim(0,13)
plt.ylim(0,1450)
error_df = pd.DataFrame(error_)
plt.grid()
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.legend()

# Saving trained model
model.save('trained_model_with_random')

# 5. Model validation

## 5.1 Comparion between predicted and semi-analytical models

In [ ]:
# Importing relevant libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Loading trained model
a_model = tf.keras.models.load_model('trained_model_with_random')

# Preparing data for comparison
X_val_1, X_val_2, y_val_1, y_val_2 = train_test_split(X_test, y_test, test_size = 0.25, shuffle = 'True')

# Cleansing data empirically
i=0
while i < len(y_val_2):
    if y_val_2[i]<=1:
        y_val_2 = np.delete(y_val_2,i)
        X_val_2 = np.delete(X_val_2,i,axis=0)
        i -= 1
    i+=1   

# Predicting results
y_val_pred = a_model.predict(X_val_2)

# Visulazing errors
err_per =[]

for item in range(len(X_val_2)):
    err = abs(y_val_pred[item][0]-y_val_2[item])/y_val_pred[item][0]*100
    err_per.append(err)
    
# Heatmap visulization of the results
# For the predicted results
plt.subplot(1,2,1)
sns.heatmap(y_val_pred,cmap="YlGnBu",yticklabels=False)
plt.ylabel('F(N)')
plt.xlabel('(a)')
plt.title('Predicted results')

# For the Semi-analytical results
plt.subplot(1,2,2)
sns.heatmap(y_val_2_plot,cmap="YlGnBu",yticklabels=False)
plt.subplots_adjust(bottom=0.1, right=1.2, top=0.9)
plt.ylabel('F(N)')
plt.xlabel('(b)')
plt.title('Semi-analytical results')

## 5.2 Validating the results against the Finite Element Method

In [ ]:
# Importing relevant libraries
import matplotlib.pyplot as plt
import numpy as np

# Obtaining results
# From semi-analytical model
semi = [7.0217,7.5095,5.5064,10.4319,26.6690,59.7973,36.3253,27.6379,20.8733,15.8355,9.3697,5.7812,3.7149,2.4769,1.7065]

# From the surrogate model (deep learning-based)
surro = [7.0289,7.4741,5.5225,10.4672,26.6741,59.7948,36.3335,27.8493,20.8972,15.7654,9.3828,5.7943,3.7128,2.4402,1.6709]

# From the Finite Element Analysis
fea = [7.0229,7.5006,5.5309,10.4590,26.6530,59.8500,36.4390,27.6200,20.8680,15.8850,9.3773,5.7753,3.7046,2.4674,1.7005]

# Visualizing the validation

plt.scatter(range(len(fea)),semi,label='Semi-analytical model')
plt.scatter(range(len(fea)),surro,label='Surrogate model',marker = '*')
plt.scatter(range(len(fea)),fea,label='FEA model',marker = 'd')
plt.legend()
plt.xlabel('Number of Samples')
plt.ylabel('F (N)')
plt.grid()

# Error for surrogate and FEA
e_surro_fea = []

for i in range(len(fea)):
    er = abs(surro[i]-fea[i])/fea[i]*100
    e_surro_fea.append(er)
    
# Error for semi-analytical and FEA

e_semi_fea = []

for i in range(len(fea)):
    er = abs(semi[i]-fea[i])/fea[i]*100
    e_semi_fea.append(er)
    
# Computing the minimum, average and maximum errors
np.min(e_semi_fea)
np.mean(e_semi_fea)
np.max(e_semi_fea)

np.min(e_surro_fea)
np.mean(e_surro_fea)
np.max(e_surro_fea)

# 6. Feature importance analysis

In [ ]:
# Importing relevant libraries
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.inspection import permutation_importance as perm
import matplotlib.pyplot as plt
import seaborn as sns


# Feature importance analysis
# Noted: different models should be used to evaluate the scoring 
# Relying solely on r2 may lead to misleading conclusion.
# Papers to be read: Tarald O. Kvalseth, "Cautionary note about R^2"
# Using R^2 with caution;; R_1^2 and R_1a^2 are good to judge the goodness of fit
# for a model.. Other r_quared should not be used solely as they can cause misleading conclusion

result_negmrs = perm(model, X_val_2, y_val_2, scoring = 'neg_root_mean_squared_error', n_repeats = 200, random_state=0)
result_r2 = perm(model, X_val_2, y_val_2, scoring = 'r2', n_repeats = 200, random_state=0)
#result_mse = perm(model, X_val_re, y_val_re, scoring = 'neg_mean_squared_error', n_repeats = 100, random_state=0)
#result_mabe = perm(model, X_val_re, y_val_re, scoring = 'neg_median_absolute_error', n_repeats = 100, random_state=0)
#result_meabe = perm(model, X_val_re, y_val_re, scoring = 'neg_mean_absolute_error', n_repeats = 100, random_state=0)
#result_eve = perm(model, X_val_re, y_val_re, scoring = 'explained_variance', n_repeats = 100, random_state=0)
# Normalize the importances
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#result_norm = scaler.fit_transform(result.importances_mean.reshape(-1,1))
#result_2_norm = scaler.fit_transform(result_2.importances_mean.reshape(-1,1))

# Bar plotting the importance results
features = np.array(['R\u2081', 'h\u2081', 'R', 'h', '\u03BE'])

# Plotting feature importances per r2
r2_sorted_idx = result_r2.importances_mean.argsort()
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(7,3.5))
ax1.boxplot(result_r2.importances[r2_sorted_idx].T, 
            vert=False)
ax1.set_yticklabels(features[r2_sorted_idx])
ax1.set_xlabel('R-squared')
ax1.set_ylabel('Features for deep learning model')
ax1.set_title('(a) Boxplot of PFI')
fig.suptitle('Permutation feature importance (PFI) per R-squared')

#plt.setp(ax1.get_yticklabels(), rotation=90)
ax1.grid('on')
sns.heatmap(result_r2.importances[r2_sorted_idx][::-1], xticklabels=199, 
            yticklabels=False, cmap = 'YlGnBu')
ax2.set_xlabel('Number of iterations')
ax2.set_title('(b) Heatmap of PFI')

#sns.heatmap(result_negmrs.importances[negmrs_sorted_idx][::-1], xticklabels=False, 
 #           yticklabels=features[negmrs_sorted_idx][::-1], cmap = 'YlGnBu')
plt.tight_layout()
plt.show()

# Plotting feature importaces per loss function
negmrs_sorted_idx = result_negmrs.importances_mean.argsort()
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6,3))
ax1.boxplot(result_negmrs.importances[negmrs_sorted_idx].T, 
            vert=False)
ax1.set_yticklabels(features[negmrs_sorted_idx])
ax1.set_xlabel('PFI per loss function')
ax1.set_ylabel('Features for deep learning model')
ax1.set_title('(a) Boxplot of PFI')
fig.suptitle('Permutation feature importance per loss function')

#plt.setp(ax1.get_yticklabels(), rotation=90)
ax1.grid('on')
sns.heatmap(result_negmrs.importances[negmrs_sorted_idx][::-1], xticklabels=199, 
            yticklabels=False, cmap = 'YlGnBu')
ax2.set_xlabel('Number of permutation')
ax2.set_title('(b) Heatmap of PFI')

#sns.heatmap(result_negmrs.importances[negmrs_sorted_idx][::-1], xticklabels=False, 
 #           yticklabels=features[negmrs_sorted_idx][::-1], cmap = 'YlGnBu')
plt.tight_layout()
plt.show()

#ax2.set
#ax2.boxplot(result_r2.importances[r2_sorted_idx].T, 
 #           vert=True, labels=features[r2_sorted_idx])
#ax2.grid('on')
#fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6,5))
plt.figure()
plt.boxplot(result_negmrs.importances[negmrs_sorted_idx].T, 
            vert=False)
plt.set_yticklabels(features[negmrs_sorted_idx])

#plt.setp(ax1.get_yticklabels(), rotation=90)
plt.grid('on')
sns.heatmap(result_negmrs.importances[negmrs_sorted_idx][::-1], xticklabels=False, 
            yticklabels=False, cmap = 'YlGnBu')
#sns.heatmap(result_negmrs.importances[negmrs_sorted_idx][::-1], xticklabels=False, 
 #           yticklabels=features[negmrs_sorted_idx][::-1], cmap = 'YlGnBu')
    
plt.tight_layout()
ax1.barh([1,2,3,4,5],result_norm.T[0])
ax2.bar([1,2,3,4,5],result_2_norm[0])
plt.tight_layout()
plt.show()
sns.heatmap(result.importances)
x_ = np.arange(100)
y_ = [item for item in result.importances_mean]
plt.bar(x_, y_)

# 7. Web-based Frameworks

## 7.1 Back-end module

In [ ]:
# Importing relevant libraries
from flask import Flask, render_template, request
import os
import pickle
import numpy as np
import tensorflow as tf

# Main body
image_folder = os.path.join('static', 'images')
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = image_folder
scaler = pickle.load(open('scaler_ran', 'rb'))
filename = os.path.join(app.config['UPLOAD_FOLDER'], 'levitated_cylinders.png')
plus_arrow = os.path.join(app.config['UPLOAD_FOLDER'], 'Plus_arrow.png')
minus_arrow = os.path.join(app.config['UPLOAD_FOLDER'], 'Minus_arrow.png')


@app.route('/')
def enter_parameters():
	return render_template('mag_in.html', Fig = 'Fig.1 - Parameters of levitated cylinders', figure_geo = filename, 
                           arrow_J_1 = minus_arrow, arrow_J = plus_arrow, 
	arrow_F = plus_arrow, J = 1, J_1 = -1, predicted_force = 11.546, R = 10, h = 10, R_1 = 10, h_1 = 10, xi = 10)
@app.route('/predict', methods = ['POST'])
def result():
	model = tf.keras.models.load_model('trained_model_with_random')
	features = [np.float32(para) for para in request.form.values()]
	model_features = np.array([[features[4], features[5], features[1], features[2], features[6]]])
	model_features = scaler.transform(model_features)
	mag_force = round(-1*features[0]*features[3]*model.predict(model_features)[0][0],3)

	return render_template('mag_in.html', Fig = 'Fig.2 - Schematic of predicted results', predicted_force = mag_force, 
                           figure_geo = filename, 
	arrow_J_1 = minus_arrow if features[3] < 0 else plus_arrow, arrow_J = minus_arrow if features[0] < 0 else plus_arrow,
	arrow_F = minus_arrow if mag_force < 0 else plus_arrow, J = features[0], R = features[1], h = features[2],
	 J_1 = features[3], R_1 = features[4], h_1 = features[5], xi = features[6])


if __name__ == '__main__':
	app.run(debug=True)

## 7.2 Front-end module

In [ ]:
<!DOCTYPE html>
<html lang = 'en'>
<head>
	<title>DEMAGFO</title>
	<meta charset = 'UTF-8'>
	<meta name = 'description' content = 'Computing magnetic forces
	between permanent magnets using machine learning'>
	<meta name = 'author' content = 'Van Tai Nguyen, Michael and Matthew from University of Queensland'>
	<meta name = 'keywords' content = 'Deep learning, magnetic force, levitation force
	magnetic cylinders'>
	<meta name = 'viewport' content = 'width=device-width, initial-scale=1.0'>
</head>
<style>
div.absolute_Fig {
	position: absolute;
	top: 60px;
	right: 300px;
	width: 350px;
	height:350px;
}

div.absolute_J_1 {
	position: absolute;
	top: 70px;
	right: 138px;
	width: 350px;
	height:350px;
}

div.absolute_J {
	position: absolute;
	top: 155px;
	right: 141px;
	width: 350px;
	height:350px;
}

div.absolute_F {
	position: absolute;
	top: -30px;
	right: -234px;
	width: 350px;
	height:350px;
}

</style>
<body style = 'background-color:#FFFFFF'>
	
	<form action='http://localhost:5000/predict' method='post'>
		<!--Input the geometrical parameters of the cylinders-->
		<!--Input the geometrical parameters of the first cylinder-->
		<h2 style = 'color:blue; background-color:#99FFFF;text-align:center;' >Computing Magnetic Forces between Permanent Magnets using a Machine Learning based Data-Driven Model</h2>
		<h4>Input the material property and geometrical parameters of the lower cylinder.</h4>
		<p>Remanence (J): <input type='text' name='J' placeholder =  '{{J}} (T)'/> </p>
		<p>Radius (R): <input type='text' name='R' placeholder = '{{R}} (mm)'/></p>
		
		<p>Height (h): <input type='text' name='h' placeholder = '{{h}} (mm)'/></p>
		

		<!--Input the geometrical parameters of the second cylinder-->
		<h4>Input the material property and geometrical parameters of the upper cylinder.</h4>
		<p>Remanence (J<sub>1</sub>): <input type='text' name='J_1'
		placeholder = '{{J_1}} (T)'/></p>
		
		<p>Radius (R<sub>1</sub>): <input type='text' name='R_1' placeholder = '{{R_1}} (mm)'/></p>
		
		<p>Height (h<sub>1</sub>): <input type='text' name='h_1' placeholder = '{{h_1}} (mm)'/></p>
		

		<!--Input the separation distance between the cylinders-->
		<h4>Input the separation distance between the cylinders.</h4>
		<p>Separation distance (&#958): <input type='text' name='&#958' placeholder = '{{xi}} (mm)'/></p>

		<p style = 'color: red;'><b>Magnetic force F: {{predicted_force}} (N)</b></p>
		
		<p><input type='Submit' value='Predict'/></p>

		<p style = 'color: #000; background-color:#CCCCFF';><i>This work has the Creative Commons license of 4.0 (CC BY 4.0) which is free to adapt and share, but not for commercial purposes. This license belongs to the Univesrity of Queensland (UQ). Authors: Nguyen, Michael and Matthew from UQ.</i></p>


	</form>
		<div class = 'absolute_Fig'> <img src = '{{figure_geo}}' alt = 'Levitated_cylinders' width = '320'
		height = '370'> 
			<p> {{Fig}} </p>
		</div>

		<div class = 'absolute_J_1'> <img src = '{{arrow_J_1}}' alt = 'arrow_J_1' width = '12'
		height = '56'> 

		<div class = 'absolute_J'> <img src = '{{arrow_J}}' alt = 'arrow_J' width = '12'
		height = '56'> 
		
		<div class = 'absolute_F'> <img src = '{{arrow_F}}' alt = 'arrow_F' width = '12'
		height = '56'> 
</body>
</html>